In [ ]:
import sqlite3
from os import path
from time import time
import pandas as pd
from sklearn.externals import joblib
from html2corpus import HTML2Corpus
from html2corpus.extractors import ReadabilityExtractor, ParagraphExtractor

In [ ]:
labels = ['2015KW44', '2015KW45', '2015KW46', '2015KW47', '2015KW48', '2015KW49', '2015KW50', '2015KW51']
files = [path.join('data', 'CurrentNews', '{}.csv'.format(label)) for label in labels]
dates = [('2015-10-26', '2015-11-02'),
         ('2015-11-02', '2015-11-09'),
         ('2015-11-09', '2015-11-16'),
         ('2015-11-16', '2015-11-23'),
         ('2015-11-23', '2015-11-30'),
         ('2015-11-30', '2015-12-07'),
         ('2015-12-07', '2015-12-14'),
         ('2015-12-14', '2015-12-21')]

db_file = path.join('..', 'Crawler', 'data', 'de_news.sqlite')
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

In [ ]:
vectorizer = joblib.load(path.join('models', 'classifier', 'Vectorizer.pkl'))
classifier = joblib.load(path.join('models', 'classifier', 'Classifier.pkl'))
ext = ReadabilityExtractor(min_len=150)

Note that the title has newlines and other shit in it...

In [ ]:
def classify(rows):
    tfidf = vectorizer.transform(rows['text'])
    return classifier.predict(tfidf)

def get_data(kw):
    query = '''SELECT site, html 
               FROM Articles 
               LEFT JOIN Sources ON Articles.source = Sources.id 
               WHERE date BETWEEN ? and ?'''
    cursor.execute(query, kw)

    data = cursor.fetchmany(100)
    
    while len(data) > 0:
        for row in data:
            site, html = row
            text = ext.extract(html)
            if len(text) > 0:
                yield { 'text': text, 'site': site }
        data = cursor.fetchmany(100)

for file, date in zip(files, dates):
    articles = pd.DataFrame([article for article in get_data(date)])
    tags = classify(articles)
    articles['tag'] = tags
    articles.loc[articles['tag'] == 1].loc[:, ['text', 'site']].to_csv(file, index=False, encoding='utf-8', sep='|')

conn.close()